In [ ]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, timedelta
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen


In [ ]:
# Reading data from csv file using pandas and excluding the values containing null values
data_weather = pd.read_csv("Districtwise_India.csv",sep="\t")
df = data_weather[data_weather['Lat'].notna()]
df.Lat = df.Lat.astype(str)
df.Long = df.Long.astype(str)

# Iterating in the rows of csv to get the latitude, longitude of the districts
for index, row in df.iterrows():
    his_data=[]
    district = row['District']
    days=5
    lat= row['Lat']
    lon= row['Long']
    
# This is an example of how the values could be of any specific place
# district = 'Kolar'
# days=5
# lat='13.137'
# lon='78.134'


# To scrape the historical data, you need to provide the url parsed by bs4, number of days to get data
    for i in range(days):
    	d = date.today() - timedelta(i)
    	quote_page="https://darksky.net/details/"+lat+','+lon+'/'+str(d)+"/us12/en"
    	page = urlopen(quote_page)
    	soup = BeautifulSoup(page, "html.parser")
     
     # components of weather to be extracted
    	precipitation_box= soup.find(attrs={"class": "precipProbability"}).find(attrs={"class":"num swip"})
    	rain_box= soup.find(attrs={"class": "precipAccum swap"}).find(attrs={"class":"num swip"})
    	temp_box = soup.find(attrs={"class": "temperature"}).find(attrs={"class":"num"})
    	wind_box = soup.find(attrs={"class": "wind"}).find(attrs={"class":"num swip"})
    	pressure_box = soup.find(attrs={"class": "pressure"}).find(attrs={"class":"num swip"})
    	humidity_box = soup.find(attrs={"class": "humidity"}).find(attrs={"class":"num swip"})
    	dew_point_box = soup.find(attrs={"class": "dew_point"}).find(attrs={"class":"num"})
    	high_point_box = soup.find(attrs={"class":"highLowTemp swip"}).find(attrs={"class": "highTemp swip"}).find(attrs={"class": "temp"})
    	low_point_box = soup.find(attrs={"class":"highLowTemp swip"}).find(attrs={"class": "lowTemp swap"}).find(attrs={"class": "temp"})

     # Storing the values in the list and using .text to extract the values
    	weather_components= [str(d),pressure_box.text,dew_point_box.text,humidity_box.text,temp_box.text,high_point_box.text.strip('/')[:2],low_point_box.text.strip('/')[:2],wind_box.text,precipitation_box.text,rain_box.text]
     
    # iterating through the list of items in weather_components to get the values and appending it to the empty list
    	for i,val in enumerate(weather_components):
    		if val=='???' or val=='-':
    		   weather_components[i]= 'No Data Found'

    	his_data.append(weather_components)
    	print(weather_components)

# To get the degree symbol in header row
    a = u"\u00b0"
  
 # creating csv header for the file and appending the data in it using writerows
    header = ['Date','Pressure(mb)',"Dew Point("+a+')','Humidity(%)', 'Temperature('+a+')','Min Temp('+a+')','Max Temp('+a+')','Wind(mph)', 'Precipitation(%)', "Rain(in)"]

    filename = district+'.csv'

# Writing header, historical data of district in files using the filename
    with open(filename, 'w') as myfile:
        wr = csv.writer(myfile, lineterminator='\n')
        wr.writerow(header)
        wr.writerows(his_data)